In [ ]:
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch

def main(df):
    # load dataset for training
    td = df.iloc[:, -40:].values  # extract only ground truth
    td = np.expand_dims(td, axis=2)

    is_noisy = df.iloc[:, 1].values.astype(np.int64)
    td = torch.tensor(td, dtype=torch.float)
    print('Using input type with shape of', td.shape)

    # define model
    net = LSTM(40).cuda()  # assuming each of the 40 columns is a feature
    print('Training detector instanced by', net.__class__.__name__)

    # load checkpoint or train from scratch
    if args.resume:
        if os.path.isfile(args.resume):
            print('=> loading checkpoint "{}"'.format(args.resume))
            checkpoint = torch.load(args.resume)
            start_epoch = checkpoint['epoch']
            net.load_state_dict(checkpoint['state_dict'])

            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})".format(args.resume, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.resume))

    train_x, test_x, train_y, test_y = train_test_split(td, is_noisy, test_size=0.1, random_state=42)
    train_dataset = TensorDataset(train_x, train_y)
    test_dataset = TensorDataset(test_x, test_y)
    train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
    test_dataloader = DataLoader(test_dataset, batch_size=512, shuffle=False, num_workers=4)

    # define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(net.parameters(), lr=0.1)
    max_epoch = 10
    best_prec = 0

    for epoch in range(max_epoch):
        net.train()
        loss_sigma = 0.0  #
        correct = 0.0
        total = 0.0
        for i, (train_data, train_label) in enumerate(train_dataloader):
            train_data, train_label = Variable(train_data).cuda(), Variable(train_label).cuda()
            out = net(train_data)

            loss = criterion(out, train_label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(out.data, 1)
            total += train_label.size(0)
            correct += (predicted == train_label).squeeze().sum().cpu().numpy()
            loss_sigma += loss.item()

        print("Training: Epoch[{:0>3}/{:0>3}]  Loss: {:.4f} Acc:{:.2%}".format(
            epoch + 1, max_epoch, loss_sigma, correct / total))

# Load your DataFrame here
df = pd.read_csv('your_file.csv')
main(df)
